In [ ]:
%load_ext zipline
%load_ext watermark
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipline
from yahoofinancials import YahooFinancials
import warnings
import pandas_datareader as pdr
plt.style.use('seaborn')

plt.rcParams['figure.figsize'] = [32, 20]
plt.rcParams['figure.dpi'] = 200
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
from scripts.get_from_pdr_to_panel import get_from_pdr_to_panel

panel = get_from_pdr_to_panel() #nvidia

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 503 (major_axis) x 5 (minor_axis)
Items axis: NVDA to NVDA
Major_axis axis: 2016-01-04 00:00:00+00:00 to 2017-12-29 00:00:00+00:00
Minor_axis axis: open to volume


E:\anaconda3\envs\env_zipline\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
from zipline.api import order, record, symbol, set_benchmark
import zipline
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

def initialize(context):
    pass


def handle_data(context, data):
    todays_pr= data.current(symbol('NVDA'),'price')
    diff= context.portfolio.cash - todays_pr 

    hist = data.history(symbol('NVDA'),'price', 35, '1d')
    sma_35= hist.mean()
    sma_5= hist[-5:].mean()
    #print(hist)
    record(sma_35=sma_35)
    record(sma_5=sma_5)
    record(NVDA=todays_pr)

    if diff>0:
        # Trading logic
        if sma_5 > sma_35:
            # order_target orders as many shares as needed to
            # achieve the desired number of shares.
            order(symbol("NVDA"), 2)
        elif short_mavg < long_mavg:
            order(symbol("NVDA"), -2)
            
    
perf = zipline.run_algorithm(start=datetime(2016, 2, 24, 0, 0, 0, 0, pytz.utc),
                      end=datetime(2017, 12, 29, 0, 0, 0, 0, pytz.utc),
                      initialize=initialize,
                      capital_base=100,
                      handle_data=handle_data,
                      data=panel)

In [5]:
perf.columns

Index(['NVDA', 'algo_volatility', 'algorithm_period_return', 'alpha',
       'benchmark_period_return', 'benchmark_volatility', 'beta',
       'capital_used', 'ending_cash', 'ending_exposure', 'ending_value',
       'excess_return', 'gross_leverage', 'long_exposure', 'long_value',
       'longs_count', 'max_drawdown', 'max_leverage', 'net_leverage', 'orders',
       'period_close', 'period_label', 'period_open', 'pnl', 'portfolio_value',
       'positions', 'returns', 'sharpe', 'short_exposure', 'short_value',
       'shorts_count', 'sma_35', 'sma_5', 'sortino', 'starting_cash',
       'starting_exposure', 'starting_value', 'trading_days', 'transactions',
       'treasury_period_return'],
      dtype='object')

In [14]:
import plotly.express as px
import plotly.graph_objects as go
fig= go.Figure()
fig.add_scatter(x=perf.index, y=perf.portfolio_value,mode='lines', name="sma_35")
fig.show()

In [13]:
import plotly.express as px

fig= go.Figure()
fig.add_scatter(x=perf.index, y=perf.sma_35,mode='lines', name="sma_35")
fig.add_scatter(x=perf.index, y=perf.sma_5,mode='lines', name="sma_5")
fig.add_scatter(x=perf.index, y=perf.NVDA,mode='lines', name="NVDA")
fig.show()

In [17]:
perf[['NVDA', 'starting_cash', 'transactions', 'algorithm_period_return']]

perf.transactions[2]

[{'amount': 2,
  'commission': None,
  'dt': Timestamp('2016-02-26 21:00:00+0000', tz='UTC'),
  'order_id': '4fb3cb6249ff43879bd1e23bba2f4e75',
  'price': 31.695840305328367,
  'sid': Equity(0 [NVDA])}]

In [18]:
perf[['NVDA', 'starting_cash', 'transactions', 'algorithm_period_return']]


,NVDA,starting_cash,transactions,algorithm_period_return
2016-02-24 21:00:00+00:00,31.820000,100.000000,[],0.000000
2016-02-25 21:00:00+00:00,31.889999,100.000000,"[{'price': 31.90594438934326, 'order_id': 'c0c...",-0.000339
2016-02-26 21:00:00+00:00,31.680000,36.186111,"[{'price': 31.695840305328367, 'order_id': '4f...",-0.004876
2016-02-29 21:00:00+00:00,31.360001,-27.207569,[],-0.017676
2016-03-01 21:00:00+00:00,32.750000,-27.207569,[],0.037924
2016-03-02 21:00:00+00:00,32.939999,-27.207569,[],0.045524
2016-03-03 21:00:00+00:00,32.650002,-27.207569,[],0.033924
2016-03-04 21:00:00+00:00,32.549999,-27.207569,[],0.029924
2016-03-07 21:00:00+00:00,32.340000,-27.207569,[],0.021524
2016-03-08 21:00:00+00:00,31.750000,-27.207569,[],-0.002076


In [19]:
perf.to_csv("temp.csv")